In [1]:
# IMPORTS
import pandas as pd

import tkinter as tk
from tkinter import filedialog as fd

import logging
import logging.config
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder

import ast

import re
from unidecode import unidecode

from datetime import datetime

import bcrypt

In [2]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

# cargar datos para conectar a AGROCITTCA
data = open(ruta + '/_keys/agrocittca.data','r')
password = data.readlines()[0]
data.close()

# extraccion de los datos
datos_cnx = ast.literal_eval(password)

# lectura de llave ssh
mypkey = paramiko.RSAKey.from_private_key_file(ruta + '/_keys/llave_db_agrocittca', password='micontrasena')

# ssh variables
ssh_host = datos_cnx['ssh_host']
ssh_port = datos_cnx['ssh_port']
localhost = datos_cnx['localhost']
localport = datos_cnx['localport']
ssh_user = datos_cnx['ssh_user']

# database variables
sql_username = datos_cnx['sql_username']
sql_password = datos_cnx['sql_password']
sql_main_database = datos_cnx['sql_main_database']

In [3]:
# funcion para ejecutar una script SQL
# estado = 0 para select
# estado = 1 para update
def dbQuery(q, estado = 0):
    """ access the database over the SSH tunnel and execute the query """
    logger = logging.getLogger()
    logger.debug("Connecting to reporting server...", end="")
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(localhost, 3306)) as tunnel:
        logger.debug("Connected")
        logger.debug("Connecting to database...")
        try:
            conn = pymysql.connect(
                host='127.0.0.1', 
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor)
            logger.debug("Connected")
            logger.debug("Executing query...")
            cursor = conn.cursor()
            cursor.execute(q)
            if estado==1:
                conn.commit()  # Commit the transaction
            data = cursor.fetchall()
            logger.debug("Done")
        finally:
            conn.close()
    return data

In [4]:
# GENERAR CODIGO SLUG, CODIGO UNICO
def abbreviate_para_slug(name):
    # Eliminar acentos y otros caracteres especiales
    name = unidecode(name)
    # Mantener solo caracteres alfanuméricos, espacios y guiones
    name = re.sub(r"[^\w\s-]", "", name)
    # Convertir a minúsculas
    name = name.lower()
    # Reemplazar espacios y guiones por un solo guión
    abbreviated = re.sub(r"[\s-]+", "-", name)
    return abbreviated.strip('-')

In [11]:
# optiene la fecha actual
now = datetime.now()
formatted_now = now.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_now)

2023-08-29 14:53:25


In [12]:
file_name = 'G:/OneDrive - Ingenio Azucarero Guabira S.A/AGRO-CITTCA/DATA PARA CARGAR/PROPIEDADES.XLSX'

In [14]:
data = pd.read_excel(file_name)
data.head(3)

,cod_prop,nom_prop,cod_ca,nom_ca,area,cuadrante,user_id,periodo_id
0,2,2 - JOSE TREJO--SEGUNDINA IRIARTE,18354,TREJO ESCALERA JOSE,0.80,359,413,14
1,4,4 - SAN JORGE--CATALA,1843,CATALA ABREGO SEGISMUNDO,44.00,359,630,14
2,5,5 - SAN FELIX--CATALA,1858,CATALA CAMPOS SEGISMUNDO,204.36,359,593,14


In [16]:
data.shape

(1459, 8)

In [29]:
# script para insertar registro de prop
INSERT INTO propiedad 
(codigo, nombre, slug, superficie_total, foto, estado, cuadrante, usuario_id, periodo_id, created_at, updated_at)
VALUES 
('2', '2 - JOSE TREJO--SEGUNDINA IRIARTE', '2-jose-trejo-2', '0.8', '/imagen/foto_propiedad.jpg', 'activo', '359', 413, 14, NOW(), NOW());

SyntaxError: invalid syntax (312947826.py, line 2)

In [31]:
for i, r in data.iterrows():
    print(r)
    prop = {
        codigo : str(r['cod_prop']), 
        nombre : str(r['nom_prop']), 
        slug = '', 
        superficie_total = str(r['area']), 
        foto = '/imagen/foto_propiedad.jpg', 
        estado = 'activo', 
        cuadrante = str(r['cuadrante']), 
        usuario_id = r['user_id'], 
        periodo_id = r['periodo_id'], 
        created_at = '', 
        updated_at
    }
    #query = 'insert into propiedad (codigo, nombre, slug, superficie_total, foto, estado, cuadrante, usuario_id, periodo_id, created_at, updated_at) values (\'{nombre}\', \'{ap_paterno}\', \'{ap_materno}\',\'{username}\', \'{email}\', \'{codigo}\', \'{telefono}\', \'{tipo}\', \'{estado}\', \'{slug}\', \'{password}\', {institucion_id}, {usuario_id}, \'{created_at}\', \'{updated_at}\')'.format(nombre=usuario['nombre'], ap_paterno=usuario['ap_paterno'], ap_materno=usuario['ap_materno'], username=usuario['usuario'], email=usuario['correo'], codigo=usuario['codigo'], telefono=usuario['telf'], tipo=usuario['tipo'], estado=usuario['estado'], slug='', password=usuario['contrasena'], institucion_id=usuario['inst'], usuario_id=usuario['usuario_id'], created_at=usuario['fecha'], updated_at=usuario['fecha'])
    #insert_row = dbQuery(query, 1)

cod_prop                                      2
nom_prop      2 - JOSE TREJO--SEGUNDINA IRIARTE
cod_ca                                    18354
nom_ca                      TREJO ESCALERA JOSE
area                                        0.8
cuadrante                                   359
user_id                                     413
periodo_id                                   14
Name: 0, dtype: object
cod_prop                             4
nom_prop         4 - SAN JORGE--CATALA
cod_ca                            1843
nom_ca        CATALA ABREGO SEGISMUNDO
area                              44.0
cuadrante                          359
user_id                            630
periodo_id                          14
Name: 1, dtype: object
cod_prop                             5
nom_prop         5 - SAN FELIX--CATALA
cod_ca                            1858
nom_ca        CATALA CAMPOS SEGISMUNDO
area                            204.36
cuadrante                          359
user_id                 